In [1]:
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

In [3]:
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
device

device(type='cuda')

In [1]:
!export XDG_RUNTIME_DIR=/tmp
!export DISPLAY=:0
!export SDL_VIDEODRIVER=dummy

In [6]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        # need to sample differently, see prioritized experience replay Schaul et al. 2016
        return np.random.choice(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
from game_gym_env import CarGameEnv
import pygame

env = CarGameEnv()
pygame.quit()

import pickle

t = ReplayMemory(10)
s = env.observation_space.sample()
for i in range(3):
    prev_s = s
    s = env.observation_space.sample()
    t.push(prev_s, env.action_space.sample(), s, 0)

# print(t.memory)

t1 = ReplayMemory(20)
t1.memory = deque(t.memory, 20)
# print(t1.memory)
t1.memory

In [8]:
class DQN(nn.Module):

    def __init__(self):
        super(DQN, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)

        self.fc_size = 64*3*1
        self.fc_image = nn.Linear(self.fc_size, 128)  # Flatten image output
        self.fc_scalar = nn.Linear(3, 32)  # 3 scalar features
        self.fc_merge = nn.Linear(128 + 32, 256)
        self.fc_out = nn.Linear(256, sum([3,2,2,2]))  # Output Q-values for each discrete action

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, image, scalar):
        # image: 1*30*15
        x = self.pool(F.relu(self.conv1(image))) # 16*30*15 -> 16*15*7
        x = self.pool(F.relu(self.conv2(x))) # 32*15*7 -> 32*7*3
        x = self.pool(F.relu(self.conv3(x))) # 64*7*3 -> 64*3*1
        x = x.view(-1, self.fc_size)
        x = F.relu(self.fc_image(x))

        s = F.relu(self.fc_scalar(scalar))
        merged = torch.cat((x, s), dim=1)
        merged = F.relu(self.fc_merge(merged))
        q_values = self.fc_out(merged)
        return torch.split(q_values, [3, 2, 2, 2], dim=1)

In [36]:
BATCH_SIZE = 2 #128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4
REPLAY_BUFF_SIZE = 20000 # a bit less than 3 games

L2_LOSS_WEIGHT = 0.01

In [ ]:
# modified from https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html
# from utils import state_batch_to_tensor
from config import *
import numpy as np
def state_batch_to_tensor(state_batch):
    images = torch.tensor(np.array([s["image"] / 255.0 for s in state_batch])).to(torch.float32).unsqueeze(1)
    scalars = torch.tensor(np.array([[s["speed"] / CAR_TOP_SPEED, 
                                      s["currLap"] / LAP_COUNT,
                                      s["boostsLeft"] / CAR_INIT_SPEED_BOOST_COUNT] 
                                    for s in state_batch])).to(torch.float32)
    return images, scalars

# from rl_hyperparams import *

policy_net = DQN().to(device)
target_net = DQN().to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True, weight_decay=L2_LOSS_WEIGHT)
memory = ReplayMemory(REPLAY_BUFF_SIZE)

steps_done = 0

def select_action(state):
    # epsilon-greedy approach
    global steps_done
    sample = np.random.uniform()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        np.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        image, scalar = state_batch_to_tensor([state])
        q_values = policy_net(image, scalar)
        with torch.no_grad():
            action = [torch.argmax(q, dim=1).detach().cpu().numpy()[0] for q in q_values]  # Take highest Q-value per action
            return action
            # return policy_net(state).max(1).indices.view(1, 1)
    else:
        return env.action_space.sample()

def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))
    
    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states_img, non_final_next_states_scr = state_batch_to_tensor([s for s in batch.next_state
                                                if s is not None])
    state_img_batch, state_scr_batch = state_batch_to_tensor(batch.state)
    reward_batch = torch.cat(batch.reward)
    action_batch = torch.cat(batch.action)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    q_values = policy_net(state_img_batch, state_scr_batch)
    state_action_values = torch.stack([
        q_values[i].gather(1, action_batch[:, i].unsqueeze(1)).squeeze(1)
        for i in range(len(q_values))
    ], dim=1).sum(dim=1)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_q_values = target_net(non_final_next_states_img, non_final_next_states_scr)
        next_state_values[non_final_mask] = torch.stack([
            q.max(1).values for q in next_q_values
        ], dim=1).sum(dim=1)
    
    # Compute the expected Q values
    next_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, next_state_action_values)

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()



In [ ]:
# import copy
if torch.cuda.is_available() or torch.backends.mps.is_available():
    num_episodes = 600
else:
    num_episodes = 50

env = CarGameEnv()
steps_done = 0

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    # state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        if t%60 == 0: print(t//60)
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action)
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = observation #torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, 
                    torch.tensor([action]),
                    next_state, 
                    reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)
        
        if done:
            # episode_durations.append(t + 1)
            # plot_durations()
            print("done")
            break
    

pygame.quit()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
done
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
done
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86

KeyboardInterrupt: 

In [35]:
import pygame
import numpy as np

In [41]:
from car_physics import Car
from checkpoint import generateCheckpoints
from utils import get_rl_state

In [54]:
from racing_game import Game
from config import *
# import copy
game = Game()
game.reset()

In [51]:
def get_state(game):
    return {
        "image":get_rl_state(game.carViewWindow, game.lowresImg, game.car),
        "speed":game.car.vel,
        # "timeLeft": TIME_LIMIT-self.game.timer,
        "currLap":(game.score+1) // len(game.checkpoints),
        "boostsLeft":game.car.boostCount
    }

In [55]:
GAME_NAME = "1"
transitionsOfGame = deque()

In [ ]:
isAccelerating = False
isBreaking = False
steerDirection = 1
tryBoosting = False

gameStarted = False # press to start
done = False

state = get_state(game)

while not done:
    if not gameStarted:
        if pygame.event.peek(pygame.KEYDOWN) \
         and pygame.key.get_pressed()[pygame.K_UP]:
            gameStarted = True
    if not gameStarted:
        game.display()
        continue

    prevState = state.copy()
    # Process player inputs.
    tryBoosting = False
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            raise SystemExit
        if event.type == pygame.MOUSEBUTTONDOWN:
            # Get mouse position
            mouse_pos = pygame.mouse.get_pos()
            print(f"Mouse clicked at: {mouse_pos}")
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT: 
                steerDirection += 1
            if event.key == pygame.K_RIGHT:
                steerDirection -= 1
            if event.key == pygame.K_UP:
                isAccelerating = True
            if event.key == pygame.K_DOWN:
                isBreaking = True
            if event.key == pygame.K_SPACE:
                tryBoosting = True
            if event.key == pygame.K_PLUS:
                game.car.BASE_ACCEL += 0.01
                print(game.car.BASE_ACCEL)
            if event.key == pygame.K_MINUS:
                game.car.BASE_ACCEL -= 0.01
                print(game.car.BASE_ACCEL)
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT: 
                steerDirection -= 1
            if event.key == pygame.K_RIGHT:
                steerDirection += 1
            if event.key == pygame.K_UP:
                isAccelerating = False
            if event.key == pygame.K_DOWN:
                isBreaking = False
    reward = game.inputProcessing(steerDirection, isAccelerating, isBreaking, tryBoosting)
    game.display(DEBUG=True)
    game.timeUpdate(framerate=FPS_RATE)

    state = get_state(game)

    if game.score >= len(game.checkpoints) * LAP_COUNT:
        reward += TIME_LIMIT - game.timer
        transitionsOfGame.append(Transition(prevState, [steerDirection, isAccelerating, isBreaking, tryBoosting], state, reward))
        pygame.quit()
        break
    
    transitionsOfGame.append(Transition(prevState, [steerDirection, isAccelerating, isBreaking, tryBoosting], state, reward))

with open("../saves/games/savegame_%s.pkl" % GAME_NAME, "wb") as f:
    pickle.dump(transitionsOfGame, f)

AttributeError: 'collections.deque' object has no attribute 'memory'

In [12]:
pygame.quit()